In [1]:
import pandas as pd
import csv

In [2]:
filtered_stopsearch = pd.read_csv(r"C:\Users\20223661\Downloads\filtered_stop_and_search.csv")
pas_boroughs = pd.read_csv(r"C:\Users\20223661\Downloads\pas_boroughs.csv")
df_borough_recent = pd.read_csv(r"C:\Users\20223661\OneDrive - TU Eindhoven\Documents\University related\Data Science Y2\JBG050 Data Challenge 2\MPS Borough Level Crime (most recent 24 months).csv")

In [3]:
pas_boroughs['good_job'] = pd.to_numeric(pas_boroughs['good_job'], errors = 'coerce')
pas_boroughs['trust'] = pd.to_numeric(pas_boroughs['trust'], errors = 'coerce')
pas_boroughs['fair_treatment'] = pd.to_numeric(pas_boroughs['fair_treatment'], errors = 'coerce')
pas_boroughs['dealing_issues'] = pd.to_numeric(pas_boroughs['dealing_issues'], errors = 'coerce')

pas_boroughs = pas_boroughs[['borough', 'trust', 'good_job', 'dealing_issues', 'fair_treatment']]
pas_boroughs_aggregated = pas_boroughs.groupby('borough').mean().reset_index()

In [4]:
# Melt the dataframe
df_melted = df_borough_recent.melt(id_vars=['MajorText', 'MinorText', 'LookUp_BoroughName'], 
                    var_name='Month', 
                    value_name='CrimeCount')

# Group by Borough and Crime Type, summing the crime counts
df_borough_grouped = df_melted.groupby(['LookUp_BoroughName', 'MajorText', 'MinorText']).agg({'CrimeCount': 'sum'}).reset_index()

# Find the crime type with the highest count per borough
idx = df_borough_grouped.groupby(['LookUp_BoroughName'])['CrimeCount'].idxmax()
df_max_crime = df_borough_grouped.loc[idx]

df_result = df_max_crime[['LookUp_BoroughName', 'MajorText', 'MinorText', 'CrimeCount']]
df_result.columns = ['Borough', 'Major Crime Type', 'Specific Crime Type', 'Total Crime Count']

# The most occuring crimes per borough
print(df_result)

                                       Borough  \
48                        Barking and Dagenham   
97                                      Barnet   
143                                     Bexley   
191                                      Brent   
238                                    Bromley   
278                                     Camden   
334                                    Croydon   
382                                     Ealing   
433                                    Enfield   
481                                  Greenwich   
529                                    Hackney   
577                     Hammersmith and Fulham   
626                                   Haringey   
677                                     Harrow   
725                                   Havering   
773                                 Hillingdon   
821                                   Hounslow   
868                                  Islington   
904                     Kensington and Chelsea   


In [5]:
# Aggregate the original crime data by borough to get the total crime counts for each borough
df_result['Total Crime Count'] = df_result.iloc[:, 3:].sum(axis=1)
df_total_crimes = df_result.groupby('Borough')['Total Crime Count'].sum().reset_index()
df_total_crimes.columns = ['borough', 'Total Crime Count']

# Merging the two dataframes on the borough name
df_borough_PAS_merged = pd.merge(df_total_crimes, pas_boroughs_aggregated, on='borough')

# Compute the correlation matrix
correlation_matrix = df_borough_PAS_merged[['Total Crime Count', 'trust', 'good_job', 'dealing_issues', 'fair_treatment']].corr()

# Display the correlation matrix
print(correlation_matrix)

                   Total Crime Count     trust  good_job  dealing_issues  \
Total Crime Count           1.000000  0.009091 -0.135122       -0.043946   
trust                       0.009091  1.000000  0.791132        0.946958   
good_job                   -0.135122  0.791132  1.000000        0.799183   
dealing_issues             -0.043946  0.946958  0.799183        1.000000   
fair_treatment             -0.113291  0.928554  0.668879        0.906423   

                   fair_treatment  
Total Crime Count       -0.113291  
trust                    0.928554  
good_job                 0.668879  
dealing_issues           0.906423  
fair_treatment           1.000000  


In [6]:
# Aggregate the number of stop searches per borough
stop_searches_per_borough = filtered_stopsearch.groupby('Borough').size().reset_index(name='stop_searches')

# Merge the aggregated stop searches with the scores data
merged_stop_PAS_df = pd.merge(pas_boroughs_aggregated, stop_searches_per_borough, how='left', left_on='borough', right_on='Borough')

merged_stop_PAS_df = merged_stop_PAS_df.drop(columns=['borough', 'Borough'])

In [7]:
# Calculate the correlation matrix
correlation_matrix1 = merged_stop_PAS_df.corr()

print(correlation_matrix1)

                   trust  good_job  dealing_issues  fair_treatment  \
trust           1.000000  0.791132        0.946958        0.928554   
good_job        0.791132  1.000000        0.799183        0.668879   
dealing_issues  0.946958  0.799183        1.000000        0.906423   
fair_treatment  0.928554  0.668879        0.906423        1.000000   
stop_searches  -0.592414 -0.562409       -0.592156       -0.605614   

                stop_searches  
trust               -0.592414  
good_job            -0.562409  
dealing_issues      -0.592156  
fair_treatment      -0.605614  
stop_searches        1.000000  
